## OpenAI Function Calling with Tool Usage

In [1]:
from openai import OpenAI
import json
from os import getenv

client = OpenAI(api_key=getenv("OPENAI_API_KEY"))

def schedule_meeting(date, time, attendees):
    # Connect to calendar service:
    return { "event_id": "1234", "status": "Meeting scheduled successfully!",
            "date": date, "time": time, "attendees": attendees }

OPENAI_FUNCTIONS = {
    "schedule_meeting": schedule_meeting
}

In [2]:
# Our predefined function JSON schema:
functions = [
    {
        "type": "function",
        "function": {
            "type": "object",
            "name": "schedule_meeting",
            "description": "Set a meeting at a specified date and time for designated attendees",
            "parameters": {
                "type": "object",
                "properties": {
                    "date": {"type": "string", "format": "date"},
                    "time": {"type": "string", "format": "time"},
                    "attendees": {"type": "array", "items": {"type": "string"}},
                },
                "required": ["date", "time", "attendees"],
            },
        },
    }
]

In [3]:
# Start the conversation:
messages = [
    {
        "role": "user",
        "content": "Schedule a meeting on 2023-11-01 at 14:00 with Alice and Bob.",
    }
]

# Send the conversation and function schema to the model:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=messages,
    tools=functions,
    tool_choice="auto"
)

response = response.choices[0].message

# Check if the model wants to call our function:
if response.tool_calls:
    # Get the first function call:
    first_tool_call = response.tool_calls[0]

    # Find the function name and function args to call:
    function_name = first_tool_call.function.name
    function_args = json.loads(first_tool_call.function.arguments)
    print("This is the function name: ", function_name)
    print("These are the function arguments: ", function_args)

    function = OPENAI_FUNCTIONS.get(function_name)

    if not function:
        raise Exception(f"Function {function_name} not found.")

    # Call the function:
    function_response = function(**function_args)

    # Share the function's response with the model:
    messages.append(
        {
            "role": "function",
            "name": "schedule_meeting",
            "content": json.dumps(function_response),
        }
    )

    # Let the model generate a user-friendly response:
    second_response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106", messages=messages
    )

    print(second_response.choices[0].message.content)

This is the function name:  schedule_meeting
These are the function arguments:  {'date': '2023-11-01', 'time': '14:00', 'attendees': ['Alice', 'Bob']}
Meeting Scheduled:

Date: 2023-11-01
Time: 14:00
Attendees: Alice, Bob

Please mark your calendars accordingly.


--------------------------------

## Parallel Function Calling:

In [4]:
# Start the conversation:
messages = [
    {
        "role": "user",
        "content": '''Schedule a meeting on 2023-11-01 at 14:00 with Alice and Bob. 
        Then I want to schedule another meeting on 2023-11-02 at 15:00 with Charlie and Dave.'''
    }
]

# Send the conversation and function schema to the model:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=messages,
    tools=functions,
)

response = response.choices[0].message

# Check if the model wants to call our function:
if response.tool_calls:
    for tool_call in response.tool_calls:
        # Get the function name and arguments to call:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        print("This is the function name: ", function_name)
        print("These are the function arguments: ", function_args)

        function = OPENAI_FUNCTIONS.get(function_name)

        if not function:
            raise Exception(f"Function {function_name} not found.")

        # Call the function:
        function_response = function(**function_args)

        # Share the function's response with the model:
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": json.dumps(function_response),
            }
        )

    # Let the model generate a user-friendly response:
    second_response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106", messages=messages
    )

    print(second_response.choices[0].message.content)

This is the function name:  schedule_meeting
These are the function arguments:  {'date': '2023-11-01', 'time': '14:00', 'attendees': ['Alice', 'Bob']}
This is the function name:  schedule_meeting
These are the function arguments:  {'date': '2023-11-02', 'time': '15:00', 'attendees': ['Charlie', 'Dave']}
Two meetings have been scheduled:

1. Meeting with Alice and Bob
   - Date: 2023-11-01
   - Time: 14:00

2. Meeting with Charlie and Dave
   - Date: 2023-11-02
   - Time: 15:00


---

## Function Calling in LangChain

In [3]:
from langchain.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Optional

class Article(BaseModel):
    """Identifying key points and contrarian views in an article."""

    points: str = Field(..., description="Key points from the article")
    contrarian_points: Optional[str] = Field(
        None, description="Any contrarian points acknowledged in the article"
    )
    author: Optional[str] = Field(None, description="Author of the article")

_EXTRACTION_TEMPLATE = """Extract and save the relevant entities mentioned \
in the following passage together with their properties.

If a property is not present and is not required in the function parameters, do not include it in the output."""

# Create a prompt telling the LLM to extract information
prompt = ChatPromptTemplate.from_messages(
    {("system", _EXTRACTION_TEMPLATE), ("user", "{input}")}
)

model = ChatOpenAI()

pydantic_schemas = [Article]

# Convert Pydantic objects to the appropriate schema:
tools = [convert_to_openai_tool(p) for p in pydantic_schemas]

# Give the model access to these tools:
model = model.bind_tools(tools=tools)

# Create an end to end chain
chain = prompt | model | PydanticToolsParser(tools=pydantic_schemas)

In [4]:
result = chain.invoke(
    {
        "input": """In the recent article titled 'AI adoption in industry', key points addressed include the growing interest in AI in various sectors, the increase in AI research, and the need for responsible AI. However, the author, Dr. Jane Smith, acknowledges a contrarian view — that without stringent regulations, AI may pose high risks."""
    }
)
print(result)

[Article(points='the growing interest in AI in various sectors, the increase in AI research, and the need for responsible AI', contrarian_points='without stringent regulations, AI may pose high risks', author='Dr. Jane Smith')]


---

## Extracting Chains with LangChain

In [9]:
from langchain.chains.openai_tools import create_extraction_chain_pydantic
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field

# Make sure to use a recent model that supports tools
model = ChatOpenAI(model="gpt-3.5-turbo-1106")

class Person(BaseModel):
    """A person's name and age."""

    name: str = Field(..., description="The person's name")
    age: int = Field(..., description="The person's age")

In [10]:
chain = create_extraction_chain_pydantic(Person, model)
chain.invoke({'input':'''Bob is 25 years old. He lives in New York. He likes to play basketball. Sarah is 30 years old. She lives in San Francisco. She likes to play tennis.'''})

[Person(name='Bob', age=25), Person(name='Sarah', age=30)]